# Odgovarjanje na vprašanja po strukturiranih podatkih 
V tem zvezku predstavimo tematiko spraševanja po strukturiranih podatkih na [zbirki NBA iger](https://www.kaggle.com/datasets/nathanlauga/nba-games). \
Naš cilj je sestaviti verigo ki prevede naše vprašanje v PostgreSQL, ga izvede in odgovori.

In [ ]:
%%capture
!pip install langchain langchain-experimental sqlalchemy python-dotenv openai psycopg2

In [ ]:
import os

import dotenv
from langchain.utilities import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain.schema.runnable import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

# from langchain.agents import AgentExecutor
from langchain_experimental.sql import SQLDatabaseChain
from langchain.schema.output_parser import NoOpOutputParser

In [ ]:
dotenv.load_dotenv(override=True)

In [ ]:
if not dotenv.load_dotenv():
    os.environ["DB_CONNECT"] = "db-uri"
    os.environ["OPENAI_API_KEY"] = "openai-api-key"

### Kaj LLM potrebuje
Da omogočimo LLM-ju iskanje po naših podatkih mu moramo podati informacije o našem okolju. Zato definiramo sledeče:
  * DB dialekt -> Postgres
  * DB struktura -> Dinamično iz DB

Oboje je podano LLM-ju preko prompta.

In [ ]:
PROMPT = """You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}
"""

postgres_prompt = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=PROMPT,
)

Inicializacija LLM-ja in povezave do baze.

In [ ]:
db = SQLDatabase.from_uri(os.getenv("DB_CONNECT"))
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

Definiramo verigo.

In [ ]:
def _strip(text: str) -> str:
    return text.strip()


inputs = {
    "input": lambda x: x["question"] + "\nSQLQuery: ",
    "top_k": lambda _: 10,
    "table_info": lambda x: db.get_table_info(table_names=x.get("table_names_to_use")),
}

# Langchain Expression Language
sql_query_chain = (
    RunnableParallel(inputs)
    | postgres_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | NoOpOutputParser()
    | _strip
)

In [ ]:
sql_query_chain.invoke(
    {
        "question": "What game did Luka Doncic have the highest three pointers percentage?"
    }
)

In [ ]:
for s in sql_query_chain.stream(
    {
        "question": "What game did Luka Doncic have the highest three pointers percentage?"
    }
):
    print(s, end="", flush=True)

V resnici ima LangChain Določene cevovode pripravljene že v naprej.

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("What game did Luka Doncic have the highest three pointers percentage in?")

In [ ]:
print(db_chain.run("What game has an ID of 22101059?"))

In [ ]:
db_chain.run("Which teams have ID's of 1610612766 and 1610612742?")

Fun but not really usefull. Lets try and make it return more human readable data.

In [ ]:
PROMPT = """You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".
Do not return ID's of teams and players, instead return their names.
When talking about a game always mention between which teams the game was played, never just respond with its ID.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}
"""
postgres_prompt = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=PROMPT,
)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=postgres_prompt, verbose=True)
# , use_query_checker=True

In [ ]:
db_chain.run("What game did Luka Doncic have the highest three pointers percentage in?")

In [ ]:
db_chain.run(
    "What game did Luka Doncic have the highest three pointers percentage in and what was it?"
)

In [ ]:
db_chain.run(
    "But what game was that?"
)

Možne izboljšave prikazanega cevovoda:
* Boljši opisi tabel in stolpcev.
* Primeri vprašanj in poivedb v pozivu (statično ali dinamično)

Kaj manjka:
* Sposobnost pogovora.
* Sposobnost popravljanja napak.
* Podpora za več SQL poizvedb.